In [ ]:
!pip -q install tfx
!pip -q install transformers
!pip -q install pillow

In [10]:
data_path = "gs://beans-lowres/tfrecords"
local_data_path = "data"

model_file = "modules/model.py"
model_fn = "modules.model.run_fn"

proprocessing_file = "modules/preprocessing.py"
preprocessing_fn = "modules.preprocessing.preprocessing_fn"

schema_file = "schema.pbtxt"

In [2]:
!mkdir {local_data_path}
!gsutil cp {data_path}/*-00-*.tfrec {local_data_path}/

Copying gs://beans-lowres/tfrecords/train-00-32.tfrec...
Copying gs://beans-lowres/tfrecords/val-00-32.tfrec...
- [2 files][ 48.0 MiB/ 48.0 MiB]                                                
Operation completed over 2 objects/48.0 MiB.                                     


In [3]:
import tfx
tfx.__version__

'1.9.1'

In [58]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import ImportExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Pusher
from tfx.proto import example_gen_pb2

In [333]:
context = InteractiveContext()

In [334]:
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name="train", pattern="train-*.tfrec"),
        example_gen_pb2.Input.Split(name="eval", pattern="val-*.tfrec"),
    ]
)
example_gen = ImportExampleGen(
    input_base=local_data_path, 
    input_config=input_config
)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [335]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [336]:
context.show(statistics_gen.outputs['statistics'])

In [337]:
%%writefile {schema_file}

feature {
  name: "image"
  type: FLOAT

  presence {
    min_fraction: 1.0
  }

  float_domain {
    min: 0
    max: 255
  }

  shape {
    dim { size: 256 }
    dim { size: 256 }
    dim { size: 3 }
  }
}

feature {
  name: "image_shape"
  type: INT

  presence {
    min_fraction: 1.0
  }

  shape {
    dim { size: 3 }
  }
}

feature {
  name: "label"
  type: INT

  presence {
    min_fraction: 1.0
  }

  int_domain {
    min: 0
    max: 2
  }

  shape {
    dim { size: 1 }
  }  
}

Overwriting schema.pbtxt


In [338]:
schema_gen = tfx.components.ImportSchemaGen(
    schema_file=schema_file)

context.run(schema_gen)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [339]:
example_validator = ExampleValidator(
  statistics=statistics_gen.outputs['statistics'],
  schema=schema_gen.outputs['schema']   
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [340]:
context.show(example_validator.outputs['anomalies'])

In [341]:
!mkdir modules

mkdir: cannot create directory ‘modules’: File exists


In [342]:
%%writefile {proprocessing_file}

import tensorflow as tf

IMAGE_KEY = "image"
LABEL_KEY = "label"

MODEL_INPUT_IMAGE_KEY = "pixel_values"
MODEL_INPUT_LABEL_KEY = "labels"

def _transformed_name(name): 
    return name + "_xf"

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
      inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
      Map from string feature key to transformed feature operations.
    """
    # print(inputs)
    outputs = {}

    features = tf.map_fn(preprocess, inputs)

    features[IMAGE_KEY] = tf.image.resize(
        features[IMAGE_KEY], [INPUT_IMG_SIZE, INPUT_IMG_SIZE]
    )

    features[IMAGE_KEY] = features[IMAGE_KEY] / 255.0
    features[IMAGE_KEY] = tf.transpose(features[IMAGE_KEY], [0, 3, 1, 2])

    outputs[MODEL_INPUT_IMAGE_KEY] = features[IMAGE_KEY]
    outputs[MODEL_INPUT_LABEL_KEY] = features[LABEL_KEY]

    return outputs

Overwriting modules/preprocessing.py


In [343]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn=preprocessing_fn)

In [344]:
context.run(transform)

{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}
{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}


{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}
{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}


{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}
{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}
{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(256, 256, 3) dtype=float32>, 'image_shape': <tf.Tensor 'map/while/TensorArrayV2Read_1/TensorListGetItem:0' shape=(3,) dtype=int64>, 'label': <tf.Tensor 'map/while/TensorArrayV2Read_2/TensorListGetItem:0' shape=(1,) dtype=int64>}
{'image': <tf.Tensor 'map/while/TensorArrayV2Read/TensorListGetItem:0' shape=(2

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [345]:
%%writefile {model_file}

from typing import List, Dict, Tuple

import absl
import tensorflow as tf
import tensorflow_transform as tft
from transformers import ViTFeatureExtractor, TFViTForImageClassification
from tfx.components.trainer.fn_args_utils import FnArgs

from tfx_bsl.tfxio import dataset_options
from tfx.components.trainer.fn_args_utils import DataAccessor

feature_extractor = ViTFeatureExtractor()

_TRAIN_LENGTH = 128
_EVAL_LENGTH = 128
_TRAIN_BATCH_SIZE = 8
_EVAL_BATCH_SIZE = 8

_EPOCHS = 1

_LABELS = ['angular_leaf_spot', 'bean_rust', 'healthy']

_CONCRETE_INPUT = "pixel_values"

def INFO(text: str):
    absl.logging.info(text)

###

MODEL_INPUT_IMAGE_KEY = "pixel_values"
MODEL_INPUT_LABEL_KEY = "labels"

def _normalize_img(
    img, mean=feature_extractor.image_mean, std=feature_extractor.image_std
):
    img = img / 255
    mean = tf.constant(mean)
    std = tf.constant(std)
    return (img - mean) / std


def _preprocess_serving(string_input):
    decoded_input = tf.io.decode_base64(string_input)
    decoded = tf.io.decode_jpeg(decoded_input, channels=3)
    resized = tf.image.resize(decoded, size=(224, 224))
    normalized = _normalize_img(resized)
    normalized = tf.transpose(
        normalized, (2, 0, 1)
    )  # Since HF models are channel-first.
    return normalized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def _preprocess_fn(string_input):
    decoded_images = tf.map_fn(
        _preprocess_serving, string_input, dtype=tf.float32, back_prop=False
    )
    return {_CONCRETE_INPUT: decoded_images}


def _model_exporter(model: tf.keras.Model):
    m_call = tf.function(model.call).get_concrete_function(
        tf.TensorSpec(
            shape=[None, 3, 224, 224], dtype=tf.float32, name=_CONCRETE_INPUT
        )
    )

    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def serving_fn(string_input):
        labels = tf.constant(list(model.config.id2label.values()), dtype=tf.string)
        images = _preprocess_fn(string_input)

        predictions = m_call(**images)
        indices = tf.argmax(predictions.logits, axis=1)
        pred_source = tf.gather(params=labels, indices=indices)
        probs = tf.nn.softmax(predictions.logits, axis=1)
        pred_confidence = tf.reduce_max(probs, axis=1)
        return {"label": pred_source, "confidence": pred_confidence}

    return serving_fn

###

def _parse_tfr(proto):
    feature_description = {
        "image": tf.io.VarLenFeature(tf.float32),
        "image_shape": tf.io.VarLenFeature(tf.int64),
        "label": tf.io.VarLenFeature(tf.int64),
    }
    rec = tf.io.parse_single_example(proto, feature_description)
    image_shape = tf.sparse.to_dense(rec["image_shape"])
    image = tf.reshape(tf.sparse.to_dense(rec["image"]), image_shape)
    label = tf.sparse.to_dense(rec["label"])
    return {"pixel_values": image, "labels": label}

def _preprocess(example_batch):
    images = example_batch["pixel_values"]
    images = tf.transpose(images, perm=[0, 1, 2, 3]) # (batch_size, height, width, num_channels)
    images = tf.image.resize(images, (224, 224))
    images = tf.transpose(images, perm=[0, 3, 1, 2])

    labels = example_batch["labels"]
    labels = tf.transpose(labels, perm=[0, 1]) # So, that TF can evaluation the shapes.
    
    return {"pixel_values": images, "labels": labels}

def _input_fn(
    file_pattern: List[str],
    data_accessor: DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    is_train: bool = False,
    batch_size: int = 200,
) -> tf.data.Dataset:
    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        dataset_options.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=MODEL_INPUT_LABEL_KEY
        ),
        tf_transform_output.transformed_metadata.schema,
    )

    return dataset

def _build_model():
  id2label={str(i): c for i, c in enumerate(_LABELS)}
  label2id={c: str(i) for i, c in enumerate(_LABELS)}

  model = TFViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(_LABELS),
    label2id=label2id,
    id2label=id2label,
  )

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer='adam', loss=loss, metrics=["accuracy"])

  return model  

def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=True,
        batch_size=_TRAIN_BATCH_SIZE,
    )

    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=False,
        batch_size=_EVAL_BATCH_SIZE,
    )

    model = _build_model()

    model.fit(
        train_dataset,
        steps_per_epoch=_TRAIN_LENGTH // _TRAIN_BATCH_SIZE,
        validation_data=eval_dataset,
        validation_steps=_EVAL_LENGTH // _EVAL_BATCH_SIZE,
        epochs=_EPOCHS,
    )

    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=_model_exporter(model)
    )

Overwriting modules/model.py


In [346]:
trainer = Trainer(
    run_fn=model_fn,
    transformed_examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
)

In [347]:
context.run(trainer)

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/bias:0', 'vit/pooler/dense/kernel:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


16/16 [==============================] - 253s 15s/step - loss: 1.1478 - accuracy: 0.3828 - val_loss: 1.1529 - val_accuracy: 0.3438


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Instructions for updating:
Use fn_output_signature instead


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))